## Depreciated!!

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import gym
from gym import spaces

# set matplotlib font to arial
plt.rcParams['font.family'] = 'Arial'




In [23]:
 # if GPU is to be used
DEVICE = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

print(f"device = {DEVICE}")

device = mps


# Defining the game environment

In [1]:
class ForagingEnv(gym.Env):
    def __init__(self, ser_lvl=0.5, grid_size=200, reward_block=(180, 180, 184, 184), penalty_block=(40, 40, 54, 54), initial_agent_pos = [100, 100]):
        super(ForagingEnv, self).__init__()

        self.grid_size = grid_size # set the grid size
        self.reward_block = reward_block # define a 4x4 reward block (x_start, y_start, x_end, y_end)
        self.penalty_block = penalty_block # define as 4x4 penalty block (x_start, y_start, x_end, y_end)
        self.state = np.zeros((self.grid_size, self.grid_size)) # initialize the state
        self.agent_pos = initial_agent_pos # set the initial agent position
        self.action_space = spaces.Discrete(4) # define the action space: 4 actions (up, down, left, right)
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.grid_size, self.grid_size), dtype=np.float32) # define the observation space
        self.serotonin_level = ser_lvl # set the initial serotonin level
        self.serotonin_level_0 = ser_lvl # set the initial serotonin level -- helpful in reset
        self.initial_agent_pos = initial_agent_pos # set the initial agent position -- helpful in reset

    def reset(self):
        """
        Reset the environment and agent's position
        """
        self.state = np.zeros((self.grid_size, self.grid_size))
        self.agent_pos = self.initial_agent_pos
        self.serotonin_level = self.serotonin_level_0
        return self.state # return the state
    
    def step(self, action):
        '''
        Take an action and update the environment.
        1. [0] controls up/down
        2. [1] controls left/right
        '''
        # update agent's position based on the action
        # up
        if action == 0:
            self.agent_pos[0] = max(0, self.agent_pos[0] - 1)
        # down
        elif action == 1:
            self.agent_pos[0] = min(self.grid_size - 1, self.agent_pos[0] + 1)

        # left
        elif action == 2:
            self.agent_pos[1] = max(0, self.agent_pos[1] - 1)

        # right
        elif action == 3:
            self.agent_pos[1] = min(self.grid_size - 1, self.agent_pos[1] + 1)

        ## set done flag
        done = False
        reward = -1 # default reward

        ## check if the agent is in the reward block
        if self.reward_block[0] <= self.agent_pos[0] <= self.reward_block[2] and self.reward_block[1] <= self.agent_pos[1] <= self.reward_block[3]:
            reward = 100
            done = True

        ## check if the agent is in the penalty block
        if self.penalty_block[0] <= self.agent_pos[0] <= self.penalty_block[2] and self.penalty_block[1] <= self.agent_pos[1] <= self.penalty_block[3]:
            reward = -100
            done = True

        ## TODO: update serotonin level if needed

        return self.state, reward, done, {}
    
    def render(self, mode = 'human'):
        '''
        Render the environment
        '''

        fig, ax = plt.subplots()
        ax.set_xlim(0, self.grid_size)
        ax.set_ylim(0, self.grid_size)

        # draw the reward block
        reward_patch = mpatches.Rectangle((self.reward_block[0], self.reward_block[1]), self.reward_block[2] - self.reward_block[0], self.reward_block[3] - self.reward_block[1], color='green')
        ax.add_patch(reward_patch)

        # draw the penalty block
        penalty_patch = mpatches.Rectangle((self.penalty_block[0], self.penalty_block[1]), self.penalty_block[2] - self.penalty_block[0], self.penalty_block[3] - self.penalty_block[1], color='red')
        ax.add_patch(penalty_patch)

        # draw the agent
        ax.plot(self.agent_pos[1], self.agent_pos[0], 'ko', label = 'Agent')

        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)

        plt.gca().invert_yaxis()
        # plt.axis('off')
        plt.grid(True)
        # plt.legend()


        # sns.despine()


        plt.show()

## test the environment

env =  ForagingEnv()
env.reset()
# env.render()

# # simulate 1 step
# env.step(1)  # Move the agent down
# env.render()  # Render the environment

# env.step(3)  # Move the agent down
# env.render()  # Render the environment

steps_arr = np.random.randint(0, 4, 100)
for step in steps_arr:
    env.step(step)

env.render()  # Render the environment


NameError: name 'gym' is not defined

In [24]:
# Defining a deep Q-network

class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.layers(x)
    
# Defining the agent

class DQNAgent:
    def __init__(self, state_dimension, action_dimension, device = DEVICE):
        self.state_dimension = state_dimension
        self.action_dimension = action_dimension
        self.q_network = DQN(state_dimension, action_dimension).to(device)
        self.target_network = DQN(state_dimension, action_dimension).to(device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=0.001, momentum=0.9)
        self.memory = deque(maxlen=10000)
        self.batch_size = 32
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.device = device
        
        